The cell below removes the annoying app bar thing at the bottom of a notebook.

In [4]:
%%javascript
$('.nbp-app-bar').toggle()

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import sqlite3
import seaborn as sns
#import pysal as ps
#from pysal.contrib.viz import mapping as maps
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomTreesEmbedding, RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix, auc, roc_curve, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import resample


%matplotlib inline

The cell below makes it so pandas will show me all the columns.

In [3]:
pd.set_option('display.max_columns', 40)

The below cell loads the spatialite extention to sqlite. I got the code from the spatialite mailing list
https://groups.google.com/d/msg/spatialite-users/o0jUwMUqx_g/OEat2JTUAAAJ
I can get it to work without geopandas. With geopandas imported it kills the ipython kernel within seconds. Ironic! Without one, the other is useless.

In [ ]:
#conn = sqlite3.connect(':memory:') 
#conn.enable_load_extension(True) 
#conn.execute("SELECT load_extension('mod_spatialite')") 

The below code for importing all the sql data came from: https://www.dataquest.io/blog/python-pandas-databases/

In [ ]:
conn = sqlite3.connect("./FPA_FOD_20170508.sqlite")

In [ ]:
#this is if you want to read the data in as tuples

#cur = conn.cursor()
#cur.execute("SELECT * FROM fires;")
#results = cur.fetchall()

In [ ]:
df_all = pd.read_sql_query("SELECT * FROM fires;", conn)
#this grabs the whole db and puts it into a df using the sqlite3.Connection

In [ ]:
df_all.shape

In [ ]:
df_all.head(2)

Since pandas can't handle the geometry information the Shape column is useless anyway, I'm going to drop it. It lives on in the original sqlite db.

In [ ]:
df_all.drop(['Shape'], axis=1, inplace=True)

In [ ]:
df_all.columns

In [ ]:
df_all.STAT_CAUSE_DESCR.unique()

There are entries in my target column that I can't use so I am dropping them.

In [ ]:
df_all.drop(df_all[df_all['STAT_CAUSE_DESCR'] == 'Missing/Undefined'].index, inplace=True)

In [ ]:
df_all.shape

In [ ]:
df_all.dtypes

In [ ]:
df_all.isnull().sum()

I'm going to create a new df, df_pred, by dropping columns from the FPA_FOD data. I'm going to read that into a csv so I have it for later, and then stop using the full set. I will keep the full set on my SSD, of course. I am making these decisions based on the data atlas located here: https://www.kaggle.com/davideanastasia/contour-map-of-us-wildfire-dataset/data

In [ ]:
df_all['FIRE_CODE'].value_counts()

I'm going to drop features that I don't care about and then do feature engineering.

In [ ]:
df_all = df_all.drop(['OBJECTID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
                  'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
                  'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
                  'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
                  'LOCAL_INCIDENT_ID', 'FIRE_NAME',
                  'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
                  'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_TIME', 
                  'STAT_CAUSE_CODE', 'CONT_TIME', 
                  'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE', 
                  'OWNER_DESCR', 'COUNTY', 'FIPS_NAME'], axis=1)

#at some point redo this. Instead of doing a drop just include all feats I want to keep. Much more readable, shorter

In [ ]:
df_all.shape

In [ ]:
df_all.head(20)

Time for feature engineering.

The 2 columns with the dates for discovery and containment are in the Julian format so I will have to convert them into gregorian timestamps and then engineer a duration. I do think day of year discovered will have some predictive power and will keep that column. I will also engineer month and a day-of-week discovered features.

In [ ]:
df_all['START_DATE'] = pd.to_datetime(df_all['DISCOVERY_DATE'], unit='D', infer_datetime_format=True, origin='julian')

In [ ]:
df_all['END_DATE'] = pd.to_datetime(df_all['CONT_DATE'], unit='D', infer_datetime_format=True, origin='julian')

In [ ]:
df_all.head(2)

In [ ]:
df_all = df_all.drop(['DISCOVERY_DATE', 'CONT_DATE'], axis=1)

In [ ]:
df_all.head(3)

In [ ]:
df_all['BURN_TIME'] = (df_all['END_DATE'] - df_all['START_DATE']).astype('timedelta64[D]')

In [ ]:
df_all['MONTH'] = pd.DatetimeIndex(df_all['START_DATE']).month

In [ ]:
df_all['DOW'] = pd.DatetimeIndex(df_all['START_DATE']).weekday_name

In [ ]:
df_all.head(3)

In [ ]:
df_all = df_all.drop(['CONT_DOY'], axis=1)

In [ ]:
df_all = df_all.drop(['END_DATE'], axis=1)

In [ ]:
df_all.head(3)

In [ ]:
df_all = df_all.drop(['FOD_ID'], axis=1)

In [ ]:
df_all.head()

In [ ]:
df_all['BURN_TIME'].isnull().sum()

In [ ]:
fig, ax = plt.subplots(figsize=(40,5))
sns.countplot(x=df_all['BURN_TIME'].value_counts(), data=df_all)

I'm going to make a new column to use as my target, "Arson". If the cause was arson the value will be 1, else 0.

In [ ]:
# the below works but it isn't vectorized and it takes a hot minute. Going to try to do something faster for practice.

# def make_arson(dataframe):
#     if dataframe['STAT_CAUSE_DESCR'] == 'Arson':
#         val = 1
#     else:
#         val = 0
#     return val
#
#
# df_all['ARSON'] = df_all.apply(make_arson, axis=1)

In [ ]:
#humbug. df_all['ARSON'] = [x for x in df_all['STAT_CAUSE_DESCR'] if x =='Arson' x = 1 else 0]

In [ ]:
df_all.loc[df_all['STAT_CAUSE_DESCR'] != 'Arson', 'ARSON'] = 0
df_all.loc[df_all['STAT_CAUSE_DESCR'] == 'Arson', 'ARSON'] = 1

df_all.head(3)

Wow that was a lot faster.

In [ ]:
df_all['ARSON'] = df_all['ARSON'].astype(int)

Now That I've made the target column for arson I can drop the original column.

In [ ]:
df_all.shape

In [ ]:
df_all.head()

In [ ]:
df_all.drop(['STAT_CAUSE_DESCR'], axis=1, inplace=True)

In [ ]:
df_all.ARSON.value_counts()

In [ ]:
281455/1713742

16.42% of these fires were started by arsonists! Crazy. Also: that's a pretty unbalanced class. Hopefully that won't cause problems. Famous last words.

Over half of all the FIRE_CODE entries are nulls. DROPPED.

In [ ]:
df_all.drop(['FIRE_CODE'], axis=1, inplace=True)

In [ ]:
df_all.isnull().sum()

I'm going to drop FIPS, because I think State will cover location and going all the way down to county risks overfitting on historic data. A county may have an arsonist in it for a time who may have been arrested, moved or died; using a coefficient for that county that was affected while the arsonist was acitive will decrease accuracy. This is also true in a state but given the differences in population between states and differenes in weather conditions I think it will be predictive and have a lower risk of over fitting.

In [ ]:
df_all.drop(['FIPS_CODE'], axis=1, inplace=True)

In [ ]:
df_all.head(3)

In [ ]:
df_all.dtypes

I don't really need to do this because I'm going to label encode and then one-hot encode it later but that will make just doing one loop for all the things that need encoding simpler, if a little more time consuming to run. Worth it.

In [ ]:
df_all['MONTH'] = df_all['MONTH'].apply(lambda x: calendar.month_name[x])

Whoops, ran it twice.

In [ ]:
df_all.head(2)

After much consideration I am going to drop the start date, keep the day-of-year, but transform it to be cyclic so the algorithms can see that 1 and 365 are actually closer than 60 and 365.

In [ ]:
df_all.drop(['START_DATE'], axis=1, inplace=True)

I need to encode my categorical values. I can use one-hot encoding or get dummies and drop one column. For month and day-of-week there are only 12 and 7 categories, respectively, so for them I'll get dummies. For the other 2 I'll use one-hot because there are so categories and I don't want to dataframe that big considering how much modeling I'm doing, the ram required would get too big.

Now I'm going to try impute data for BURN_TIME. I want to test to see if the data is missing at random. I'm going to check for a correlation between BURN_TIME and the other features.

In [ ]:
df_all.head(2)

In this notebook I'm going to just drop the missing burn time values and then include it in my analysis. See what that gets me.

In [ ]:
df_wburn = df_all.drop(df_all[df_all['BURN_TIME'].isnull() == True].index)

In [ ]:
df_wburn.shape

In [ ]:
df_wburn.STATE.nunique()

In [ ]:
df_wburn.head(20)

In [ ]:
df_wburn.isnull().sum()

In [ ]:
df_wburn.dtypes

I want to make the DISCOVERED_DOY cyclic.

In [ ]:
df_wburn.shape

In [ ]:
df_wburn['DOY_SIN'] = np.sin(df_wburn.DISCOVERY_DOY*(2.*np.pi/365))
df_wburn['DOY_COS'] = np.cos(df_wburn.DISCOVERY_DOY*(2.*np.pi/365))

In [ ]:
df_wburn.shape

In [ ]:
df_wburn.head(2)

In [ ]:
df_wburn = df_wburn.drop(['DISCOVERY_DOY'], axis=1)

In [ ]:
df_wburn.shape

In [ ]:
df_wburn.head(2)

I need to scale this. There are a ton of outliers so I'm going to use RobustScaler.

In [ ]:
#robsca = RobustScaler()

In [ ]:
#df_wburn['BURN_TIME'] = robsca.fit_transform(df_wburn['BURN_TIME'].values.reshape(-1, 1))

In [ ]:
#df_wburn.head(10)

Turns out that with all the zeros in BURN_TIME it's a sparse matrix so robust scalar won't work. Sparse data often calls for MaxAbsScaler but that scales by the highest absolute value, and since there are outliers in both BURN_TIME and FIRE_SIZE I don't want to do that.

In [ ]:
ss = StandardScaler()

In [ ]:
df_wburn['BURN_TIME'] = ss.fit_transform(df_wburn['BURN_TIME'].values.reshape(-1, 1))

In [ ]:
df_wburn.shape

In [ ]:
df_wburn.head(3)

In [ ]:
plt.hist(df_wburn.FIRE_SIZE)

In [ ]:
df_wburn.shape

In [ ]:
robsca = RobustScaler()

In [ ]:
df_wburn['FIRE_SIZE'] = robsca.fit_transform(df_wburn['FIRE_SIZE'].values.reshape(-1, 1))

In [ ]:
df_wburn.head(30)

In [ ]:
df_wburn.shape

In [ ]:
df_wburn.to_csv('./data_to_encode.csv', index=False)

In [ ]:
df_wburn = pd.read_csv('./data_to_encode.csv')
print(df_wburn.shape)

OKAY. Now everything that needed to be is scaled and normalized. Time to encode my categorical values.

THE BELOW WAS THE ORIGINAL PLAN BUT I SWITCHED TO ONE-HOT ENCODING EVERYTHING AND USING PCA TO REDUCE DIMENSIONALITY.

I need to drop 1 dummy variable for each of the features I just got dummies from. I'm choosing Wednesday because it's the middle of the weekdays, of which there are 5 vs just 2 weekends (I believe that the ratio of causes that fires started on weekend days had will be different from weekdays. I am hoping this will help with interpretation.) I'm choosing January essentially at random, it's the first value which is what get_dummies has as a parameter.

In [ ]:
# categorical_vars=['MONTH','DOW']
# for var in categorical_vars:
#     cat_list='var'+'_'+var
#     cat_list = pd.get_dummies(df_all[var], prefix=var, drop_first=False)
#     df_all1=df_all.join(cat_list)
#     df_all=df_all1

#DECIDED TO DO ONE-HOT ENCODING ON EVERYTHING AND THEN USE PCA TO DEAL WITH THE COLLINIERITY PROBLEM.

In [ ]:
lab_enc = LabelEncoder()
oh_enc = OneHotEncoder()

need_encoding = [df_wburn['STATE'], df_wburn['MONTH'], df_wburn['DOW']]

for feat in need_encoding:
    feat_int_encoded = lab_enc.fit_transform(feat)
    feat_int_encoded_reshape = feat_int_encoded.reshape(len(feat_int_encoded), 1) 
    feat_onehot_encoded = oh_enc.fit_transform(feat_int_encoded_reshape)
    df_feat = pd.DataFrame(feat_onehot_encoded.toarray())
    df_wburn2 = pd.merge(df_wburn, df_feat, left_index=True, right_index=True)
    df_wburn = df_wburn2

df_wburn.shape

In [ ]:
df_wburn.head()

In [ ]:
df_wburn.columns

In [ ]:
df_wburn.head(3)

In [ ]:
df_wburn.tail(3)

In [ ]:
df_wburn.STATE.nunique()

In [ ]:
df_wburn.DOW.nunique()

In [ ]:
df_wburn.MONTH.nunique()

In [ ]:
df_wburn.drop(['STATE', 'MONTH', 'DOW'], axis=1, inplace=True)

In [ ]:
df_wburn.shape

In [ ]:
df_wburn.isnull().sum().sum()

I wrote the dataframe to .csv as this is what I'm going to perform PCA on and I wanted it as a file.

In [ ]:
df_wburn.to_csv('./final_df.csv', index=False)

In [5]:
df_wburn = pd.read_csv('./final_df.csv')

Futher down you can see there's a strange phenomenon where the first PCA feature after transformation accounts for over 99.99% of the variation. In the next cells I tried to discover why that might be.

In [ ]:
print(np.corrcoef(df_wburn['FIRE_SIZE'], df_wburn['BURN_TIME']))

That's a nice looking dataframe for machine learning. To deal with the all that dimensionality and collinearity I'm going to use PCA.

In [ ]:
for i in df_wburn.columns:
    print(np.corrcoef(df_wburn['ARSON'], df_wburn[i])[0,1])

I was thinking that one un-transformed feature might account for nearly all the variation so I checked corrcoeffs above. It's actually nicely distributed. The largest one is 0.3, which is not strong.

Below I looked to see if having both DOY_SIN and DOY_COS in the data was causing the weirdness. It was not.

In [ ]:
df_nosin = df_wburn.drop(['DOY_SIN'], axis=1)

In [ ]:
df_nosin.head(3)

In [ ]:
pca_nosin = PCA()
pca_nosin.fit(df_nosin.drop(['ARSON'], axis=1))

In [ ]:
var_exp_nosin = pca_nosin.explained_variance_ratio_
cum_var_exp_nosin = np.cumsum(var_exp_nosin)

In [ ]:
len(pca_nosin.components_)

In [ ]:
plt.figure(figsize=(15,10))

# HERE IS THE BASIC PLOT

# Plot the cumulative explained variance
component_number = list(range(1, 75))
plt.plot(component_number, cum_var_exp_nosin, lw=3)

# NOW MAKE IT LOOK PRETTY

# Add horizontal lines at y=0 and y=100
plt.axhline(y=0, linewidth=5, color='grey', ls='dashed')
plt.axhline(y=1, linewidth=3, color='grey', ls='dashed')

# Set the x and y axis limits
ax = plt.gca()
ax.set_xlim([1,74])
ax.set_ylim([-0.05,1.05])

# Label the axes
ax.set_ylabel('cumulative variance explained', fontsize=16)
ax.set_xlabel('component', fontsize=16)

# Make the tick labels bigger
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
    
# Add title
ax.set_title('component vs cumulative variance explained\n', fontsize=20)

plt.show()

In [ ]:
print(cum_var_exp_nosin)

Taking out DOY_SIN actually increased the percentage of variation accounted for by the first PCA feature. So I went back to using the full dataframe. At this point, all the categorical data has been one-hot encoded, the PCA transformation has taken care of the collinearity that encoding causes, the numerical data has all been scaled, no correlation coefficients were very strong, and taking out one of the DOY columns basically did nothing. I conclude that something **extremely** improbable has occured, and because I have a good grasp on statistics I know that happens. I am moving on to do a PCA transformation on the entire data set.

In [ ]:
pca_wburn = PCA()
pca_wburn.fit(df_wburn.drop(['ARSON'], axis=1))

The next few cells are from the PCA lesson.

In [ ]:
var_exp_wburn = pca_wburn.explained_variance_ratio_
cum_var_exp_wburn = np.cumsum(var_exp_wburn)

In [ ]:
len(pca_wburn.components_)

In [ ]:
plt.figure(figsize=(15,10))

# HERE IS THE BASIC PLOT

# Plot the cumulative explained variance
component_number = list(range(1, 76))
plt.plot(component_number, cum_var_exp_wburn, lw=3)

# NOW MAKE IT LOOK PRETTY

# Add horizontal lines at y=0 and y=100
plt.axhline(y=0, linewidth=5, color='grey', ls='dashed')
plt.axhline(y=1, linewidth=3, color='grey', ls='dashed')

# Set the x and y axis limits
ax = plt.gca()
ax.set_xlim([1,74])
ax.set_ylim([-0.05,1.05])

# Label the axes
ax.set_ylabel('cumulative variance explained', fontsize=16)
ax.set_xlabel('component', fontsize=16)

# Make the tick labels bigger
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(12) 
    
# Add title
ax.set_title('component vs cumulative variance explained\n', fontsize=20)

plt.show()

I don't trust that result at all. But I have tried a bunch of different ways to catch something going wrong with it. I dropped one of my cyclical DOY columns - nothing. I scaled and normalized both my numeric columns: fire size and burn time - nothing. I checked the corrcoeffs of each feature against Arson: the biggest one is 0.3 - nothing there either. I checked to see what the correlation between BURN_TIME and FIRE_SIZE is, it's 0.1 - nothing. I have looked over the code, which is quite simple and would be hard to screw up in such a way that it would still run, several times and found no errors. I suppose it's possible that something extraordinarily improbable has occurred here.

In [ ]:
print(cum_var_exp_wburn)

I'll keep 4 of my pca components for now. That keeps 99.999984% of my variance explained while dropping almost all of my variables. This solves the high dimensionality my data had after one-hot encoding. It also is going to make everything run much faster than if I had needed to keep a large number of features. After I've gotten to the point where I'm scoring models I'll try a few other numbers of PCA features to use, all low to keep my dimensionality down.

**UPDATE:** Setled on 6 PCA features after looking at scores for several different numbers of kept features on a logistic regression.

Time to make my training and testing sets. Then I'll transform with PCA and use that.

In [6]:
X = df_wburn.drop(['ARSON'], axis=1)
y = df_wburn['ARSON']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
pca_6 = PCA(n_components=6)
pca_6.fit(X_train)
Xt_train = pca_6.transform(X_train)
Xt_test = pca_6.transform(X_test)

About half of the code for these models came from : http://scikit-learn.org/stable/auto_examples/ensemble/plot_feature_transformation.html#sphx-glr-auto-examples-ensemble-plot-feature-transformation-py

**Logistic regression.**

In [9]:
lr = LogisticRegression()
lr.fit(Xt_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
lr.score(Xt_test, y_test)

0.839936340794759

**Number of estimators for Random Trees, Random Forest, and Gradient Boosting Classifier.** 10 is a common choice so I went with that. This will be tuned later.

In [11]:
n_estimator = 10

**Random Trees Embedding piped to Logistic Regression.** This was not part of my original plan but I found it online and it looked cool.

In [12]:
rt = RandomTreesEmbedding(max_depth=4, n_estimators=n_estimator,
    random_state=42)

rt_lm = LogisticRegression()
pipeline = make_pipeline(rt, rt_lm)
pipeline.fit(Xt_train, y_train)

Pipeline(memory=None,
     steps=[('randomtreesembedding', RandomTreesEmbedding(max_depth=4, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           random_state=42, sp...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [13]:
pipeline.score(Xt_test, y_test)

0.839936340794759

**Random Forest.**

In [14]:
rf = RandomForestClassifier(max_depth=4, n_estimators=n_estimator)
rf.fit(Xt_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
rf.score(Xt_test, y_test)

0.839936340794759

**Gradient Boosting Classifier**

In [16]:
grd = GradientBoostingClassifier(n_estimators=n_estimator)
grd.fit(Xt_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [17]:
grd.score(Xt_test, y_test)

0.839936340794759

**XGBoost.**

In [18]:
xgb = XGBClassifier()
xgb.fit(Xt_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [19]:
xgb.score(Xt_test, y_test)

/Users/ryanmetz/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8402470064367741

**Gaussian Naive Bayes.** For all binary features a Bernoulli kernel is used, for categorical data, a multinomial. My data is neither so I went with a Gaussian kernel.

In [20]:
gnb = GaussianNB()
gnb.fit(Xt_train, y_train)

GaussianNB(priors=None)

In [21]:
gnb.score(Xt_test, y_test)

0.617020116962889

**Calibrated Classifier with LinearSVC.** LinearSVC doesn't have a predict_probas method. Calibrated Classifer not only improves it's performance, it has a predict_probas method, so I use it here. It requires an extra split. This model's computational demands increase dramatically with dimensionality so I went down to using only 2 PCA features for it.

In [22]:
X_train_svc, X_test_svc, y_train_svc, y_test_svc = train_test_split(X, y, test_size=0.8, random_state=42)

In [23]:
pca2 = PCA(n_components=2)
pca2.fit(X_train_svc)
Xt_train_svc = pca2.transform(X_train_svc)
Xt_test_svc = pca2.transform(X_test_svc)

In [24]:
lsvc = LinearSVC(random_state=0)

In [25]:
cal_class_lsvc = CalibratedClassifierCV(lsvc, cv=2, method='sigmoid')
cal_class_lsvc.fit(Xt_train_svc, y_train_svc)

/Users/ryanmetz/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/calibration.py:435: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/ryanmetz/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/calibration.py:445: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/ryanmetz/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/calibration.py:447: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
/Users/ryanmetz/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/calibration.py:435: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/ryanmetz/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/calibration.py:435: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])
/Users/ryanmetz/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/calibration.py:445: R

CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
            cv=2, method='sigmoid')

In [26]:
cal_class_lsvc.score(Xt_test_svc, y_test_svc)

0.8409444248385695

The below cell was run and an ROC curve was generated below. It was terrible so I commented this all out so it wouldn't be re-run if I end up restarting this kernel and going through everything again.

In [ ]:
# Xt_train_small, Xt_train_lr, y_train_small, y_train_lr = train_test_split(Xt_train,
#                                                             y_train,
#                                                             test_size=0.5)

# grd_enc_lm = GradientBoostingClassifier(n_estimators=n_estimator)
# grd_lm = LogisticRegression()
# grd_enc = OneHotEncoder()

# grd.fit(Xt_train_small, y_train_small)
# grd_enc.fit(grd.apply(Xt_train_small)[:, :, 0])
# grd_lm.fit(grd_enc.transform(grd.apply(Xt_train_lr)[:, :, 0]), y_train_lr)

In [ ]:
# clf = SVC()
# clf.fit(Xt_train, y_train) commented out because this takes waaaaay to long to run on my laptop.

In [ ]:
#clf.score(Xt_test, y_test)

In [ ]:
# commented this out because I'm going to use roc2() from now on

# def roc(probas, true, step=0.01):
    
#     probas = probas[:,1]  
#     true = true.values 
#     assert(len(probas) == len(true))
    
#     TPRs = [] # Setting up empty list of True Positive Rate
#     FPRs = [] # Setting up empty list of False Positive Rate
    
#     for i in np.arange(0.0,1.0,step): # np.arange allows us to use step sizes that are decimals
#         preds_class = probas > i # Numpy arrays have a feature called 'broadcasting.' Check the documentation: https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html to see what this does.
#         TP = 0 
#         FP = 0
#         TN = 0
#         FN = 0
#         for index in range(len(preds_class)): # We're comparing each prediction with each true value here

#             if preds_class[index] == 1 and true[index] == 1:
#                 TP += 1
#             elif preds_class[index] == 1 and true[index] == 0:
#                 FP += 1
#             elif preds_class[index] == 0 and true[index] == 0:
#                 TN += 1 
#             elif preds_class[index] == 0 and true[index] == 1:
#                 FN += 1
                
#         TPR = TP/(TP + FN) # Calculating TPR and FPR and appending to our lists
#         FPR = FP/(FP + TN)
        
#         TPRs.append(TPR)
#         FPRs.append(FPR)
         
#     plt.rcParams['font.size'] = 14
#     plt.plot(FPRs, TPRs, color="orange")
#     plt.plot([0,1],[0,1], color='navy', lw=2, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.0])
#     plt.title('Receiver Operating Characteristic')
#     plt.xlabel("False Positive Rate")
#     plt.ylabel("True Positive Rate")
#     plt.show();

In [ ]:
#roc(lr.predict_proba(Xt_test), y_test)

In [ ]:
#roc(pipeline.predict_proba(Xt_test), y_test)

In [ ]:
#roc(rf.predict_proba(Xt_test), y_test)

In [ ]:
#roc(grd.predict_proba(Xt_test), y_test)

In [ ]:
#roc(xgb.predict_proba(Xt_test), y_test)

In [ ]:
#commented out so it can't be run again. This was terrible so I'm not going to use it, but if I go back and run
#everything again I don't want to spend the few minutes this took to run, but I don't want to lose the graph.
#roc(grd_lm.predict_proba(grd_enc.transform(grd.apply(X_test)[:, :, 0])), y_test)

Wow definitely don't use that one. I'm going to comment grd_lm out above.

The code in the next 2 cells also comes from a GA lesson.

In [27]:
def roc2(probas, true, step=0.01, plot=True):
    """
    probas should be a numpy array of predict_probas
    true is a pandas series of true labels
    step is the step size for checking thresholds
    plot is a boolean for whether we want to generate a plot
    """
    
    probas = probas[:,1]  # The output of predict_proba() is an array of the probabilities for every class, but we only want the probabilities for class 1
    true = true.values    # We need to convert the class labels from a Pandas Series to a numpy array. We do this using the .values attribute
    assert(len(probas) == len(true)) # We're making sure that our probabilities vector is the same length as our true class labesl vector
    
    TPRs = [] # Setting up empty list of True Positive Rate
    FPRs = [] # Setting up empty list of False Positive Rate
    
    for i in np.arange(0.0,1.0,step): # np.arange allows us to use step sizes that are decimals
        preds_class = probas > i # Numpy arrays have a feature called 'broadcasting.' Check the documentation: https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html to see what this does.
        TP = 0 
        FP = 0
        TN = 0
        FN = 0
        for index in range(len(preds_class)): # We're comparing each prediction with each true value here

            if preds_class[index] == 1 and true[index] == 1:
                TP += 1
            elif preds_class[index] == 1 and true[index] == 0:
                FP += 1
            elif preds_class[index] == 0 and true[index] == 0:
                TN += 1 
            elif preds_class[index] == 0 and true[index] == 1:
                FN += 1
                
        TPR = TP/(TP + FN) # Calculating TPR and FPR and appending to our lists
        FPR = FP/(FP + TN)
        
        TPRs.append(TPR)
        FPRs.append(FPR)
        
    if plot: # Only generate a plot if plot == True
        
        plt.rcParams['font.size'] = 14
        plt.plot(FPRs, TPRs, color="orange")
        plt.plot([0,1],[0,1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.title('Receiver Operating Characteristic')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.show();
        
    return np.array(TPRs), np.array(FPRs) # We'll neet to have our TPRs and FPRs as arrays so we can use broadcasting in the next function.

def sen_spec(probas, true, stat="sensitivity", stat_val=0.5, step=0.01):
    '''
    This is the function that the assignment is asking for. We'll feed in our probas and true vectors as we do with the previous function, as well
    as we are looking at, as well as the value of that stat that we want to find, and the step size for our roc function.
    
    '''
    
    stat = stat.lower() # turn the string into lowercase for ease of comparison
    
    assert(stat in ["sensitivity", "specificity"]) # Throw an error if the user did not write sensitivity or specificity
    
    TPRs, FPRs = roc2(probas, true, step=step, plot=False) # use our roc function to get vectors of TPR and FPR
    
    if stat == "sensitivity":
        diffs = np.absolute(TPRs - stat_val) # We won't necessarily be able to find a threshold that gives the exact value we're looking for, but we can find the one that gets us the closest.
        loc = np.argmin(diffs) # We want to find the index of the TPR that was closest to the value we specified
        threshold = loc * step
        sensitivity = TPRs[loc]
        spec_corr = (1 - FPRs[loc])
        
        print("Sensitivity: {:.5f}%, Specificity: {:.5f}%, Threshold: {}%".format(sensitivity*100, spec_corr*100, threshold*100))
        return TPRs, FPRs
    
    elif stat == "specificity":
        specificities = 1 - FPRs # FPR = 1 - specificity --> specificity = 1 - FPR
        diffs = np.absolute(specificities - stat_val)
        loc = np.argmin(diffs)
        threshold = loc * step
        specificity = specificities[loc]
        sens_corr = (TPRs[loc])
        
        print("Specificity: {:.5f}%, Sensitivity: {:.5f}%, Threshold: {}%".format(specificity*100, sens_corr*100, threshold*100))
        return TPRs, FPRs

In [28]:
def auc(y, x):
    
    auc = 0
    for i in range(1,len(x)):
        auc += ((y[i] + y[i-1])/2)*abs((x[i] - x[i-1]))
        
    return auc

Going to re-run all the models for scoring with the above functions.

(array([1.        , 0.9999319 , 0.99982975, 0.99965949, 0.99918278,
        0.99897848, 0.99850177, 0.99775266, 0.9360188 , 0.86951784,
        0.83093844, 0.79402751, 0.7652547 , 0.73672024, 0.70512122,
        0.67253473, 0.6426042 , 0.62190139, 0.59316263, 0.55900981,
        0.52131572, 0.47664124, 0.43700627, 0.39951648, 0.37547671,
        0.3493258 , 0.32242577, 0.28987333, 0.25565241, 0.21731136,
        0.16957232, 0.11546581, 0.05621765, 0.02557205, 0.00769545,
        0.0023495 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [29]:
roc2(lr.predict_proba(Xt_test), y_test)

In [ ]:
#TPRs_lr, FPRs_lr = sen_spec(probas=lr.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)
#this was with 5 pca feats


In [ ]:
#TPRs_lr, FPRs_lr = sen_spec(probas=lr.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)
# 7 PCA feats

In [ ]:
#TPRs_lr, FPRs_lr = sen_spec(probas=lr.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)
# 8 pca feats

A "by-hand gridsearch" shows that 6 PCA features gives the best sensitivity. Could be a local maxima but I'm not willing to make the data array even bigger than 8 given how many rows I have.

In [30]:
TPRs_lr, FPRs_lr = sen_spec(probas=lr.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)
#with 6 pca feats

KeyboardInterrupt: 

In [ ]:
auc(TPRs_lr, FPRs_lr)

In [ ]:
lr_preds = lr.predict(Xt_test)

In [ ]:
print(classification_report(y_test, lr_preds))


Bad bad bad. Gotta balance. No way around it.

This "pipeline" was a random trees embedding with a logistic regression performed on that output.

In [ ]:
roc2(pipeline.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_rt, FPRs_rt = sen_spec(probas=pipeline.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_rt, FPRs_rt)

Now for the random forest.

In [ ]:
roc2(rf.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_rf, FPRs_rf = sen_spec(probas=rf.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_rf, FPRs_rf)

Next is Gradient Boosted Trees Classifier

In [ ]:
roc2(grd.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_grd, FPRs_grd = sen_spec(probas=grd.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_grd, FPRs_grd)

Now for XGBoost

In [ ]:
roc2(xgb.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_xgb, FPRs_xgb = sen_spec(probas=xgb.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_xgb, FPRs_xgb)

Now Gaussian Naive Bayes Classifier.

In [ ]:
roc2(gnb.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_gnb, FPRs_gnb = sen_spec(probas=gnb.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_gnb, FPRs_gnb)

In [ ]:
# pca_1 = PCA(n_components=1)
# pca_1.fit(X_train)
# Xt_train_svm = pca_1.transform(X_train)
# Xt_test_svm = pca_1.transform(X_test)

In [ ]:
# clf = SVC(probability=True)
# clf.fit(Xt_train_svm, y_train)

In [ ]:
# roc2(clf.predict_proba(Xt_test), y_test)

In [ ]:
# TPRs_svc, FPRs_svc = sen_spec(probas=svc.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)


In [ ]:
# auc(TPRs_svc, FPRs_svc)

SVC is too computationally demanding for this laptop without CUDA compiling and running it on my GPU. Going to have to do that in the cloud.

In [ ]:
roc2(calclass_lsvc.predict_proba(Xt_test_svc), y_test)

I screwed up by calibrating and training on the same data set, which is fatal to this technique. I'll have to go fix that.

In [ ]:
TPRs_calcla_lsvc, FPRs_gnb = sen_spec(probas=calclass_lsvc.predict_proba(Xt_test_svc), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)


It would appear that having my classes unbalanced, cases of Arson make up only 16% of the instances in my sample. Has truly come back to haunt me. I will now deal with that in various ways.

In [ ]:
df_wburn.ARSON.value_counts()

In [ ]:
# Method 1: downsample majority class

df_minority = df_wburn[df_wburn['ARSON']==1]
df_majority = df_wburn[df_wburn['ARSON']==0]
 
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=59948,     # to match minority class
                                 random_state=42) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.ARSON.value_counts()

In [ ]:
Xd = df_downsampled.drop(['ARSON'], axis=1)
yd = df_downsampled['ARSON']

In [ ]:
X_train_down, X_test_down, y_train_down, y_test_down = train_test_split(Xd, yd, test_size=0.2, random_state=42)

In [ ]:
pca_6_down = PCA(n_components=6)
pca_6_down.fit(X_train_down)
Xdt_train = pca_6_down.transform(X_train_down)
Xdt_test = pca_6_down.transform(X_test_down)

Initially I tested on down-sampled data. This is wrong. I need to train on my downsampled data and then test against the full data set. Rather and doubling the number of cells below this I went back and re-did it the proper way.

In [ ]:
lr_down = LogisticRegression()
lr_down.fit(Xdt_train, y_train_down)

In [ ]:
lr_down.score(Xt_test, y_test)

In [ ]:
rt_down = RandomTreesEmbedding(max_depth=4, n_estimators=n_estimator,
    random_state=42)

rt_lm_down = LogisticRegression()
pipeline_down = make_pipeline(rt_down, rt_lm_down)
pipeline_down.fit(Xdt_train, y_train_down)

In [ ]:
rf_down = RandomForestClassifier(max_depth=4, n_estimators=n_estimator)
rf_down.fit(Xdt_train, y_train_down)

In [ ]:
grd_down = GradientBoostingClassifier(n_estimators=n_estimator)
grd_down.fit(Xdt_train, y_train_down)

In [ ]:
xgb_down = XGBClassifier()
xgb_down.fit(Xdt_train, y_train_down)

In [ ]:
gnb_down = GaussianNB()
gnb.fit(Xdt_train, y_train_down)

In [ ]:
lsvc_down = LinearSVC(fit_intercept=False, random_state=42)
lsvc_down.fit(Xdt_train, y_train_down)

In [ ]:
roc2(lr_down.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_lr_down, FPRs_lr_down = sen_spec(probas=lr_down.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_lr_down, FPRs_lr_down)

Not exactly the sensitivity score I was looking for. AUC-ROC score is at 76.6%, not really a change.

In [ ]:
lr_down_preds = lr_down.predict(Xt_test)

In [ ]:
roc2(pipeline_down.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_pipeline_down, FPRs_pipeline_down = sen_spec(probas=pipeline_down.predict_proba(Xt_test), true=y_test, stat="sensitivity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_pipeline_down, FPRs_pipeline_down)

That actually got worse. Something is wrong.

In [ ]:
pipeline_down_preds = pipeline_down.predict(Xt_test)

In [ ]:
print((Xdt_train).shape)
print((Xdt_test).shape)
print((y_train_down).value_counts())
print((y_test_down).value_counts())

The sizes and value counts are right. I'm not sure what's going on here. I can't find a mistake.

In [ ]:
roc2(rf_down.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_rf_down, FPRs_rf_down = sen_spec(probas=rf_down.predict_proba(Xt_test), true=y_test, stat="sensitivity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_rf_down, FPRs_rf_down)

Interesting the downsampled data provided an rf with 1% higher sensitivity.

In [ ]:
rf_down_preds = rf_down.predict(Xt_test)

In [ ]:
roc2(grd_down.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_grd_down, FPRs_grd_down = sen_spec(probas=grd_down.predict_proba(Xt_test), true=y_test, stat="specificity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_grd_down, FPRs_grd_down)

In [ ]:
grd_down_preds = grd_down.predict(Xt_test)

In [ ]:
roc2(xgb_down.predict_proba(Xt_test), y_test)

In [ ]:
TPRs_xgb_down, FPRs_xgb_down = sen_spec(probas=xgb_down.predict_proba(Xt_test), true=y_test, stat="sensitivity", stat_val = 0.99, step=0.01)


In [ ]:
auc(TPRs_xgb_down, FPRs_xgb_down)

In [ ]:
xgb_down_preds = xgb_down.predict(Xt_test)

In [ ]:
print(classification_report(y_test, lr_down_preds))
print(classification_report(y_test, pipeline_down_preds))
print(classification_report(y_test, rf_down_preds))
print(classification_report(y_test, grd_down_preds))
print(classification_report(y_test, xgb_down_preds))

**Class Balance Method 2:** up-sampling the minority class.

In [ ]:
df_wburn.ARSON.value_counts()

In [ ]:
df_majority_up = df_wburn[df_wburn.ARSON==0]
df_minority_up = df_wburn[df_wburn.ARSON==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority_up, 
                                 replace=True,     # sample with replacement
                                 n_samples=355013,    # to match majority class
                                 random_state=42) # reproducible results
 
# Combine majority class with upsampled minority class
df_up = pd.concat([df_majority_up, df_minority_upsampled])
 
# Display new class counts
df_up.ARSON.value_counts()

In [ ]:
Xu = df_up.drop(['ARSON'], axis=1)
yu = df_up['ARSON']

In [ ]:
X_train_u, X_test_u, yu_train, yu_test = train_test_split(Xu, yu, test_size=0.2, random_state=42)

In [ ]:
pca_6_up = PCA(n_components=6)
pca_6_up.fit(X_train_u)
Xu_train = pca_6_up.transform(X_train_u)
Xu_test = pca_6_up.transform(X_test_u)